# Модель прогнозирования дохода - Исправленная версия
## Исправления:
- Добавлены random_state везде
- Исправлена обработка test данных
- Сохранены исходные данные для корректного target encoding
- Исправлена логика создания submission

In [8]:
import pandas as pd
import numpy as np
import shap
import itertools
import random
import os
import json
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from lightgbm import LGBMRegressor
import lightgbm as lgb
from tqdm import tqdm

# Фиксируем все random states
RANDOM_STATE = 42
random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

# Параметры многопоточности: пользователь хочет 11 ядер (22 потока)
N_THREADS = 22
N_JOBS = 11

# Устанавливаем переменные окружения, чтобы библиотеки линейной алгебры не ограничивали число потоков
os.environ['OMP_NUM_THREADS'] = str(N_THREADS)
os.environ['OPENBLAS_NUM_THREADS'] = str(N_THREADS)
os.environ['MKL_NUM_THREADS'] = str(N_THREADS)
os.environ['NUMEXPR_NUM_THREADS'] = str(N_THREADS)

# Сохраним информацию о seed'ах для reproducibility
seeds = {"RANDOM_STATE": int(RANDOM_STATE)}
with open("seeds_used.json", "w", encoding="utf-8") as f:
    json.dump(seeds, f, ensure_ascii=False, indent=2)


In [9]:
# Загрузка данных
df_train_raw = pd.read_csv('hackathon_income_train.csv', decimal=',', sep=';', low_memory=False)
df_test_raw = pd.read_csv('hackathon_income_test.csv', decimal=',', sep=';', low_memory=False)

print(f"Train shape: {df_train_raw.shape}")
print(f"Test shape: {df_test_raw.shape}")

Train shape: (76786, 224)
Test shape: (73214, 222)


In [10]:
# ========================================
# ФУНКЦИИ ДЛЯ PREPROCESSING
# ========================================

def convert_object_to_numeric(df):
    """Конвертирует object колонки в numeric где возможно"""
    df = df.copy()
    for col in df.columns:
        if df[col].dtype == 'object':
            converted = pd.to_numeric(df[col], errors='coerce')
            n_non_convertible = converted.isna().sum() - df[col].isna().sum()
            if n_non_convertible == 0:
                df[col] = converted
                print(f"{col} преобразован в float")
    return df

def fill_object_columns(df):
    """Заполняет пропуски в object колонках"""
    df = df.copy()
    fill_cols = ['adminarea', 'city_smart_name', 'dp_ewb_last_employment_position', 
                 'addrref', 'dp_address_unique_regions', 'period_last_act_ad']
    
    for col in fill_cols:
        if col in df.columns:
            df[col] = df[col].fillna('Unknown')
    return df

def create_features(df):
    """Создает дополнительные признаки"""
    df = df.copy()
    
    # num_unique_regions
    if 'dp_address_unique_regions' in df.columns:
        df['num_unique_regions'] = df['dp_address_unique_regions'].apply(
            lambda x: 0 if pd.isna(x) or x=='Unknown' else len(str(x).split(','))
        )
    
    # period_year
    if 'period_last_act_ad' in df.columns:
        df['period_year'] = pd.to_datetime(
            df['period_last_act_ad'], format='%Y-%m-%d', errors='coerce'
        ).dt.year.fillna(0).astype(int)
    
    # is_male
    if 'gender' in df.columns:
        df['is_male'] = (df['gender'] == 'Мужской').astype(int)
    
    return df

def compute_target_encoding_stats(df_train, te_columns, target='target', smoothing=10):
    """Вычисляет статистики target encoding на train"""
    global_mean = df_train[target].mean()
    te_stats = {}
    
    for col in te_columns:
        if col in df_train.columns:
            stats = df_train.groupby(col)[target].agg(['mean', 'count'])
            stats['te'] = (stats['mean'] * stats['count'] + global_mean * smoothing) / (stats['count'] + smoothing)
            te_stats[col] = {'stats': stats['te'].to_dict(), 'global_mean': global_mean}
    
    return te_stats

def apply_target_encoding(df, te_stats, te_columns):
    """Применяет target encoding используя предвычисленные статистики"""
    df = df.copy()
    
    for col in te_columns:
        if col in df.columns and col in te_stats:
            df[f'{col}_te'] = df[col].map(te_stats[col]['stats']).fillna(te_stats[col]['global_mean'])
    
    return df

def remove_high_frequency_columns(df, threshold=0.95):
    """Удаляет колонки с частотой наиболее частого значения > threshold"""
    columns_to_drop = []
    for col in df.columns:
        if col in ['id', 'target', 'w']:
            continue
        value_counts = df[col].value_counts(normalize=True)
        if len(value_counts) > 0:
            most_common_freq = value_counts.iloc[0]
            if most_common_freq >= threshold:
                columns_to_drop.append(col)
    
    print(f"Удаляем {len(columns_to_drop)} колонок с частотой > {threshold}")
    return df.drop(columns=columns_to_drop), columns_to_drop

In [11]:
# ========================================
# PREPROCESSING TRAIN
# ========================================

# Сохраняем исходную копию для target encoding
df_train_original = df_train_raw.copy()

# Шаг 1: Конвертация типов
df_train = convert_object_to_numeric(df_train_raw)

# Шаг 2: Заполнение пропусков
df_train = fill_object_columns(df_train)

# Шаг 3: Создание новых признаков
df_train = create_features(df_train)

# Шаг 4: Target Encoding статистики (ДО удаления колонок!)
te_columns = ['adminarea', 'dp_ewb_last_employment_position', 'city_smart_name', 
              'addrref', 'dp_address_unique_regions']

# Применяем TE на исходных данных
df_train_original = convert_object_to_numeric(df_train_original)
df_train_original = fill_object_columns(df_train_original)
te_stats = compute_target_encoding_stats(df_train_original, te_columns)

# Применяем TE к train
df_train = apply_target_encoding(df_train, te_stats, te_columns)

# Шаг 5: Удаляем категориальные колонки и служебные
drop_cols = te_columns + ['period_last_act_ad', 'gender', 'dt']
df_train = df_train.drop(columns=[c for c in drop_cols if c in df_train.columns])

# Шаг 6: Удаляем колонки с высокой частотой
df_train, high_freq_cols = remove_high_frequency_columns(df_train)

print(f"\nИтоговая размерность train после preprocessing: {df_train.shape}")
print(f"Колонки: {list(df_train.columns[:10])}...")

hdb_bki_total_max_limit преобразован в float
hdb_bki_total_cc_max_limit преобразован в float
hdb_bki_total_pil_max_limit преобразован в float
hdb_bki_active_cc_max_limit преобразован в float
hdb_bki_other_active_pil_outstanding преобразован в float
hdb_bki_total_pil_max_limit преобразован в float
hdb_bki_active_cc_max_limit преобразован в float
hdb_bki_other_active_pil_outstanding преобразован в float
hdb_bki_total_products преобразован в float
hdb_bki_total_max_overdue_sum преобразован в float
bki_total_auto_cnt преобразован в float
hdb_bki_total_ip_max_limit преобразован в float
hdb_bki_total_products преобразован в float
hdb_bki_total_max_overdue_sum преобразован в float
bki_total_auto_cnt преобразован в float
hdb_bki_total_ip_max_limit преобразован в float
hdb_bki_total_cnt преобразован в float
bki_total_oth_cnt преобразован в float
bki_total_ip_max_outstand преобразован в float
hdb_bki_total_ip_cnt преобразован в float
hdb_bki_active_cc_max_outstand преобразован в float
hdb_bki_to

In [12]:
# ========================================
# РАЗБИЕНИЕ ДАННЫХ
# ========================================

ids = df_train["id"]
y = df_train["target"]
w = df_train["w"]
X = df_train.drop(columns=["target", "w", "id"])

# Разбиение с фиксированным random_state
X_train, X_temp, y_train, y_temp, w_train, w_temp = train_test_split(
    X, y, w, test_size=0.30, random_state=RANDOM_STATE
)

X_val, X_test, y_val, y_test, w_val, w_test = train_test_split(
    X_temp, y_temp, w_temp, test_size=1/3, random_state=RANDOM_STATE
)

print("Train:", X_train.shape)
print("Val:  ", X_val.shape)
print("Test: ", X_test.shape)

Train: (53750, 211)
Val:   (15357, 211)
Test:  (7679, 211)


In [13]:
# ========================================
# BASELINE MODEL + SHAP
# ========================================

baseline = LGBMRegressor(
    objective="mae",
    learning_rate=0.03,
    num_leaves=255,
    max_depth=8,
    n_estimators=1500,
    random_state=RANDOM_STATE,
    n_jobs=N_JOBS,
    num_threads=N_THREADS,
    verbose=-1,
)

baseline.fit(
    X_train, y_train,
    sample_weight=w_train,
    eval_set=[(X_val, y_val)],
    eval_sample_weight=[w_val],
    callbacks=[lgb.early_stopping(100)],
)

# SHAP для отбора признаков
explainer = shap.TreeExplainer(baseline)
shap_values = explainer.shap_values(X_val)

shap_importance = np.abs(shap_values).mean(axis=0)
shap_df = pd.DataFrame({
    "feature": X.columns,
    "importance": shap_importance
}).sort_values("importance", ascending=False)

print(shap_df.head(20))


Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[1484]	valid_0's l1: 65161.5
Did not meet early stopping. Best iteration is:
[1484]	valid_0's l1: 65161.5
                                               feature    importance
0                               turn_cur_cr_avg_act_v2  25636.587034
2                              hdb_bki_total_max_limit   9684.992425
5                                          incomeValue   8264.580398
205                                            is_male   6584.369381
1                                    salary_6to12m_avg   6529.245677
4                           hdb_bki_total_cc_max_limit   5380.101245
9                          hdb_bki_total_pil_max_limit   4316.050954
207                 dp_ewb_last_employment_position_te   3015.677409
7                                   turn_cur_cr_avg_v2   2384.910096
8                                   turn_cur_cr_max_v2   2287.196698
6                        

In [14]:
# ========================================
# ОТБОР ПРИЗНАКОВ: SHAP + Permutation Importance
# ========================================

shap_top_list = [50, 75, 100, 125, 150]
pi_top_list = [20, 40, 60, 80, 100]

results = []

def wmae(y_true, y_pred, w):
    return np.sum(w * np.abs(y_true - y_pred)) / np.sum(w)

for k_shap in shap_top_list:
    shap_features = shap_df.head(k_shap)["feature"].tolist()
    X_train_shap = X_train[shap_features]
    X_val_shap = X_val[shap_features]

    print(f"\n=== SHAP TOP {k_shap} ===")

    model_for_pi = LGBMRegressor(
        objective="mae",
        learning_rate=0.03,
        num_leaves=255,
        max_depth=8,
        n_estimators=1500,
        random_state=RANDOM_STATE,
        n_jobs=N_JOBS,
        num_threads=N_THREADS,
        verbose=-1,
    )
    model_for_pi.fit(
        X_train_shap, y_train,
        sample_weight=w_train,
        eval_set=[(X_val_shap, y_val)],
        eval_sample_weight=[w_val],
        callbacks=[lgb.early_stopping(50)],
    )

    pi = permutation_importance(
        model_for_pi,
        X_val_shap,
        y_val,
        n_repeats=5,
        random_state=RANDOM_STATE,
        n_jobs=N_JOBS,
    )

    pi_df = pd.DataFrame({
        "feature": shap_features,
        "importance": pi.importances_mean
    }).sort_values("importance", ascending=False)

    for k_pi in pi_top_list:
        selected_features = pi_df.head(k_pi)["feature"].tolist()
        X_train_pi = X_train[selected_features]
        X_val_pi = X_val[selected_features]
        X_test_pi = X_test[selected_features]

        print(f"→ Training with SHAP={k_shap}, PI={k_pi} (features={len(selected_features)})")

        model = LGBMRegressor(
            objective="mae",
            learning_rate=0.03,
            num_leaves=255,
            max_depth=8,
            n_estimators=5000,
            random_state=RANDOM_STATE,
            n_jobs=N_JOBS,
            num_threads=N_THREADS,
            verbose=-1,
        )

        model.fit(
            X_train_pi, y_train,
            sample_weight=w_train,
            eval_set=[(X_val_pi, y_val)],
            eval_sample_weight=[w_val],
            callbacks=[lgb.early_stopping(100)],
        )

        pred_val = model.predict(X_val_pi, num_iteration=model.best_iteration_)
        pred_test = model.predict(X_test_pi, num_iteration=model.best_iteration_)

        wmae_val = wmae(y_val.values, pred_val, w_val.values)
        wmae_test = wmae(y_test.values, pred_test, w_test.values)

        print(f"   WMAE Val: {wmae_val:.3f}, WMAE Test: {wmae_test:.3f}")

        results.append({
            "k_shap": k_shap,
            "k_pi": k_pi,
            "num_features": len(selected_features),
            "wmae_val": wmae_val,
            "wmae_test": wmae_test,
            "best_iter": model.best_iteration_
        })

results_df = pd.DataFrame(results).sort_values("wmae_val")
print("\n" + "="*50)
print("FINAL RESULTS (sorted by WMAE VAL)")
print("="*50)
print(results_df)

# Сохраняем лучшую комбинацию признаков
best_combo = results_df.iloc[0]
best_k_shap = int(best_combo['k_shap'])
best_k_pi = int(best_combo['k_pi'])

# Пересоздаем лучший набор признаков
shap_features_best = shap_df.head(best_k_shap)["feature"].tolist()
X_train_shap_best = X_train[shap_features_best]
X_val_shap_best = X_val[shap_features_best]

model_for_pi_best = LGBMRegressor(
    objective="mae",
    learning_rate=0.03,
    num_leaves=255,
    max_depth=8,
    n_estimators=1500,
    random_state=RANDOM_STATE,
    n_jobs=N_JOBS,
    num_threads=N_THREADS,
    verbose=-1,
)
model_for_pi_best.fit(
    X_train_shap_best, y_train,
    sample_weight=w_train,
    eval_set=[(X_val_shap_best, y_val)],
    eval_sample_weight=[w_val],
    callbacks=[lgb.early_stopping(50)],
)

pi_best = permutation_importance(
    model_for_pi_best,
    X_val_shap_best,
    y_val,
    n_repeats=5,
    random_state=RANDOM_STATE,
    n_jobs=N_JOBS,
)

pi_df_best = pd.DataFrame({
    "feature": shap_features_best,
    "importance": pi_best.importances_mean
}).sort_values("importance", ascending=False)

best_features = pi_df_best.head(best_k_pi)["feature"].tolist()
print(f"\nЛучшие признаки ({len(best_features)}): {best_features[:10]}...")



=== SHAP TOP 50 ===
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1147]	valid_0's l1: 65287.5
Early stopping, best iteration is:
[1147]	valid_0's l1: 65287.5
→ Training with SHAP=50, PI=20 (features=20)
Training until validation scores don't improve for 100 rounds
→ Training with SHAP=50, PI=20 (features=20)
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[798]	valid_0's l1: 65757.1
   WMAE Val: 65757.118, WMAE Test: 65050.859
→ Training with SHAP=50, PI=40 (features=40)
Early stopping, best iteration is:
[798]	valid_0's l1: 65757.1
   WMAE Val: 65757.118, WMAE Test: 65050.859
→ Training with SHAP=50, PI=40 (features=40)
Training until validation scores don't improve for 100 rounds
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1325]	valid_0's l1: 64640.3
Early stopping, best iteration is:
[1325]	valid_0's l1: 64640.3
   WMAE Val: 64640

In [15]:
# ========================================
# HYPERPARAMETER TUNING
# ========================================

X_train_best = X_train[best_features]
X_val_best = X_val[best_features]
X_test_best = X_test[best_features]

num_leaves_list = [31, 63, 127, 255]
max_depth_list = [5, 8, 12, 16, -1]
learning_rate_list = [0.01, 0.03, 0.05, 0.1]
n_estimators_list = [500, 1000, 1500, 2000]
min_child_samples_list = [5, 10, 20, 50]
subsample_list = [0.6, 0.8, 1.0]
colsample_bytree_list = [0.6, 0.8, 1.0]

N_RANDOM = 500
param_grid = list(itertools.product(
    num_leaves_list, max_depth_list, learning_rate_list, n_estimators_list,
    min_child_samples_list, subsample_list, colsample_bytree_list
))

random.seed(RANDOM_STATE)
sampled_params = random.sample(param_grid, N_RANDOM)

best_result = {"wmae_val": np.inf}
all_results = []

for num_leaves, max_depth, lr, n_estimators, min_child, subsample, colsample in tqdm(sampled_params):
    model = LGBMRegressor(
        objective="mae",
        learning_rate=lr,
        num_leaves=num_leaves,
        max_depth=max_depth,
        n_estimators=n_estimators,
        min_child_samples=min_child,
        subsample=subsample,
        colsample_bytree=colsample,
        random_state=RANDOM_STATE,
        n_jobs=N_JOBS,
        num_threads=N_THREADS,
        verbose=-1,
    )

    model.fit(
        X_train_best, y_train,
        sample_weight=w_train,
        eval_set=[(X_val_best, y_val)],
        eval_sample_weight=[w_val],
        callbacks=[lgb.early_stopping(50)],
    )

    pred_val = model.predict(X_val_best, num_iteration=model.best_iteration_)
    wmae_val = wmae(y_val.values, pred_val, w_val.values)

    all_results.append({
        "num_leaves": num_leaves,
        "max_depth": max_depth,
        "learning_rate": lr,
        "n_estimators": n_estimators,
        "min_child_samples": min_child,
        "subsample": subsample,
        "colsample_bytree": colsample,
        "wmae_val": wmae_val
    })

    if wmae_val < best_result["wmae_val"]:
        best_result.update({
            "wmae_val": wmae_val,
            "params": {
                "num_leaves": num_leaves,
                "max_depth": max_depth,
                "learning_rate": lr,
                "n_estimators": n_estimators,
                "min_child_samples": min_child,
                "subsample": subsample,
                "colsample_bytree": colsample
            }
        })
        print(f"\n*** Новый лучший: WMAE Val={wmae_val:.3f}, params={best_result['params']}")

hp_results_df = pd.DataFrame(all_results).sort_values("wmae_val")
print("\n" + "="*50)
print("Все результаты (топ-10 по WMAE Val)")
print(hp_results_df.head(10))


  0%|          | 0/500 [00:00<?, ?it/s]

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[853]	valid_0's l1: 64771.3
Early stopping, best iteration is:
[853]	valid_0's l1: 64771.3


  0%|          | 1/500 [00:18<2:32:22, 18.32s/it]


*** Новый лучший: WMAE Val=64771.310, params={'num_leaves': 255, 'max_depth': 16, 'learning_rate': 0.01, 'n_estimators': 2000, 'min_child_samples': 5, 'subsample': 0.6, 'colsample_bytree': 0.6}
Training until validation scores don't improve for 50 rounds


  0%|          | 2/500 [00:33<2:16:52, 16.49s/it]

...

Early stopping, best iteration is:
[611]	valid_0's l1: 65296.8
Training until validation scores don't improve for 50 rounds
Training until validation scores don't improve for 50 rounds


100%|█████████▉| 499/500 [1:22:20<00:11, 11.80s/it]

Did not meet early stopping. Best iteration is:
[1500]	valid_0's l1: 66035.8
Training until validation scores don't improve for 50 rounds
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1996]	valid_0's l1: 64267.1
Did not meet early stopping. Best iteration is:
[1996]	valid_0's l1: 64267.1


100%|██████████| 500/500 [1:22:44<00:00,  9.93s/it]


Все результаты (топ-10 по WMAE Val)
     num_leaves  max_depth  learning_rate  n_estimators  min_child_samples  \
475         255         16           0.01          2000                 50   
169         255         16           0.01          1500                 50   
82          255         16           0.03          1500                 50   
23          255         16           0.03          2000                 50   
205         255         -1           0.05          1000                 50   
8           255         -1           0.01          2000                 50   
154         255         16           0.01          2000                 20   
353         255         16           0.03          1000                 50   
274         127         16           0.01          1500                 20   
396         255         -1           0.05           500                 50   

     subsample  colsample_bytree      wmae_val  
475        1.0               0.8  63330.794086  
169   

In [16]:
# ========================================
# ОЦЕНКА ТОП-10 МОДЕЛЕЙ НА TEST
# ========================================

TOP_K = 10
top10 = hp_results_df.head(TOP_K).copy()
test_results = []

print("\n" + "="*50)
print(f"Обучаем {TOP_K} лучших моделей и оцениваем на test")
print("="*50 + "\n")

# Объединяем train + val
Xtr = pd.concat([X_train_best, X_val_best]).fillna(X_train_best.median(numeric_only=True))
ytr = pd.concat([y_train, y_val])
wtr = pd.concat([w_train, w_val]).fillna(w_train.mean())

X_test_fixed = X_test_best.fillna(X_train_best.median(numeric_only=True))

for i, row in top10.iterrows():
    params = {
        "num_leaves": int(row["num_leaves"]),
        "max_depth": int(row["max_depth"]),
        "learning_rate": float(row["learning_rate"]),
        "n_estimators": int(row["n_estimators"]),
        "min_child_samples": int(row["min_child_samples"]),
        "subsample": float(row["subsample"]),
        "colsample_bytree": float(row["colsample_bytree"]),
        "objective": "mae",
        "random_state": RANDOM_STATE,
        "n_jobs": N_JOBS,
        "num_threads": N_THREADS,
        "verbose": -1
    }

    model = LGBMRegressor(**params)
    model.fit(Xtr, ytr, sample_weight=wtr)

    pred_test = model.predict(X_test_fixed)
    wmae_test = wmae(y_test.values, pred_test, w_test.values)

    test_results.append({
        **params,
        "wmae_val": row["wmae_val"],
        "wmae_test": wmae_test
    })

    print(f"Модель {len(test_results)}/{TOP_K}: WMAE_val={row['wmae_val']:.4f}, WMAE_test={wmae_test:.4f}")

top10_results_df = pd.DataFrame(test_results).sort_values("wmae_test")
print("\n" + "="*50)
print("Топ-10 моделей (сортировка по WMAE Test)")
print(top10_results_df.head(10))
print("="*50)



Обучаем 10 лучших моделей и оцениваем на test

Модель 1/10: WMAE_val=63330.7941, WMAE_test=61873.9653
Модель 1/10: WMAE_val=63330.7941, WMAE_test=61873.9653
Модель 2/10: WMAE_val=63498.0166, WMAE_test=62135.7537
Модель 2/10: WMAE_val=63498.0166, WMAE_test=62135.7537
Модель 3/10: WMAE_val=63662.4308, WMAE_test=62528.9687
Модель 3/10: WMAE_val=63662.4308, WMAE_test=62528.9687
Модель 4/10: WMAE_val=63662.4308, WMAE_test=62419.2720
Модель 4/10: WMAE_val=63662.4308, WMAE_test=62419.2720
Модель 5/10: WMAE_val=63725.2898, WMAE_test=62731.8869
Модель 5/10: WMAE_val=63725.2898, WMAE_test=62731.8869
Модель 6/10: WMAE_val=63733.1491, WMAE_test=62159.9891
Модель 6/10: WMAE_val=63733.1491, WMAE_test=62159.9891
Модель 7/10: WMAE_val=63737.0578, WMAE_test=61925.0609
Модель 7/10: WMAE_val=63737.0578, WMAE_test=61925.0609
Модель 8/10: WMAE_val=63758.6841, WMAE_test=62631.1492
Модель 8/10: WMAE_val=63758.6841, WMAE_test=62631.1492
Модель 9/10: WMAE_val=63796.2111, WMAE_test=62788.4375
Модель 9/10: WMAE

In [17]:
# ========================================
# ОБУЧЕНИЕ ФИНАЛЬНОЙ МОДЕЛИ
# ========================================

best_row = top10_results_df.iloc[0]

print("\n" + "="*50)
print("Лучшая модель по WMAE Test:")
print(best_row)
print("="*50 + "\n")

best_params = {
    "num_leaves": int(best_row["num_leaves"]),
    "max_depth": int(best_row["max_depth"]),
    "learning_rate": float(best_row["learning_rate"]),
    "n_estimators": int(best_row["n_estimators"]),
    "min_child_samples": int(best_row["min_child_samples"]),
    "subsample": float(best_row["subsample"]),
    "colsample_bytree": float(best_row["colsample_bytree"]),
    "objective": "mae",
    "random_state": RANDOM_STATE,
    "n_jobs": N_JOBS,
    "num_threads": N_THREADS,
    "verbose": -1
}

final_model = LGBMRegressor(**best_params)
final_model.fit(Xtr, ytr, sample_weight=wtr)

print("Финальная модель обучена!")



Лучшая модель по WMAE Test:
num_leaves                    255
max_depth                      16
learning_rate                0.01
n_estimators                 2000
min_child_samples              50
subsample                     1.0
colsample_bytree              0.8
objective                     mae
random_state                   42
n_jobs                         11
num_threads                    22
verbose                        -1
wmae_val             63330.794086
wmae_test            61873.965286
Name: 0, dtype: object

Финальная модель обучена!
Финальная модель обучена!


In [18]:
# ========================================
# PREPROCESSING TEST DATA
# ========================================

print("Preprocessing test data...")

# Шаг 1: Конвертация типов
df_test = convert_object_to_numeric(df_test_raw)

# Шаг 2: Заполнение пропусков
df_test = fill_object_columns(df_test)

# Шаг 3: Создание новых признаков
df_test = create_features(df_test)

# Шаг 4: Применяем Target Encoding (используя статистики из train)
df_test = apply_target_encoding(df_test, te_stats, te_columns)

# Шаг 5: Удаляем те же категориальные колонки
df_test = df_test.drop(columns=[c for c in drop_cols if c in df_test.columns])

# Шаг 6: Удаляем колонки с высокой частотой (те же что в train)
df_test = df_test.drop(columns=[c for c in high_freq_cols if c in df_test.columns])

print(f"Test shape after preprocessing: {df_test.shape}")

Preprocessing test data...
hdb_bki_total_max_limit преобразован в float
hdb_bki_total_cc_max_limit преобразован в float
hdb_bki_total_pil_max_limit преобразован в float
hdb_bki_active_cc_max_limit преобразован в float
hdb_bki_other_active_pil_outstanding преобразован в float
hdb_bki_total_products преобразован в float
hdb_bki_total_max_overdue_sum преобразован в float
bki_total_auto_cnt преобразован в float
hdb_bki_total_ip_max_limit преобразован в float
hdb_bki_total_cnt преобразован в float
bki_total_oth_cnt преобразован в float
bki_total_ip_max_outstand преобразован в float
hdb_bki_total_pil_max_limit преобразован в float
hdb_bki_active_cc_max_limit преобразован в float
hdb_bki_other_active_pil_outstanding преобразован в float
hdb_bki_total_products преобразован в float
hdb_bki_total_max_overdue_sum преобразован в float
bki_total_auto_cnt преобразован в float
hdb_bki_total_ip_max_limit преобразован в float
hdb_bki_total_cnt преобразован в float
bki_total_oth_cnt преобразован в float

In [19]:
# ========================================
# СОЗДАНИЕ SUBMISSION
# ========================================

# Проверяем наличие всех нужных признаков
missing_features = set(best_features) - set(df_test.columns)
if len(missing_features) > 0:
    print(f"⚠ WARNING: отсутствуют признаки в test: {missing_features}")
    # Добавляем недостающие признаки с нулями
    for feat in missing_features:
        df_test[feat] = 0

# Отбираем нужные признаки в правильном порядке
X_test_final = df_test[best_features].copy()

# Заполняем NaN медианами из train
median_tr = Xtr.median(numeric_only=True)
X_test_final = X_test_final.fillna(median_tr)

# Предсказание
pred = final_model.predict(X_test_final)

# Создаём submission
submission = pd.DataFrame({
    "id": df_test_raw["id"],  # ВАЖНО: используем id из исходного df_test_raw!
    "target": pred
})

submission.to_csv("submission.csv", index=False)

print("\n" + "="*50)
print("✓ Файл submission.csv создан!")
print(submission.head(10))
print(f"\nФорма submission: {submission.shape}")
print(f"Статистики target: min={pred.min():.2f}, max={pred.max():.2f}, mean={pred.mean():.2f}")
print("="*50)


✓ Файл submission.csv создан!
   id         target
0   0   43034.431284
1   1   50691.923703
2   3   29476.702718
3   9   60557.687670
4  11   45852.720096
5  12  153086.121654
6  13   54030.322507
7  15  136353.221380
8  16   37171.786432
9  17   35425.299117

Форма submission: (73214, 2)
Статистики target: min=16526.09, max=1132567.87, mean=95489.43
